<a href="https://colab.research.google.com/github/PyBlin/Study/blob/main/PyML/Chapter4_Classification/Chap4_5_GBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.GBM( Gradient Boosting Machine )

## 5.1 GBM Intro

* Boosting : 여러 약한 학습기를 순차적으로 학습/예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나갑니다.

* AdaBoost( Adaptive Boosting )
>* 오류 데이터에 가중치를 부여하면서 부스팅을 수행하는 대표적인 알고리즘입니다.

* Gradient Boost
>* AdaBoost와 유사하나, 경사하강법( Gradient Descent )을 이용합니다.
>* 오류값 = 실제값 - 예측값
>* h(x) = y - F(x)
>* 이 오류 식을 최소화하는 방향으로 반복적으로 가중치를 업데이트하는 것이 경사하강법입니다.
>* 경사하강법은 ML에서 중요합니다.

* CART 기반 알고리즘처럼 분류와 회귀 둘 다 가능합니다.

In [1]:
from google.colab import drive
drive.mount('/drive')

path = '/drive/MyDrive/Colab/ML_Study/Chapter4_Classification/human_activity/'

Mounted at /drive


In [2]:
def get_new_feature_name_df(old_name):
    feature_dup_df = pd.DataFrame(data=old_name.groupby('column_name').cumcount(), 
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_name = pd.merge(old_name.reset_index(), feature_dup_df, how='outer')
    new_name['column_name'] = new_name[['column_name', 
                                        'dup_cnt']].apply(
                                            lambda x: x[0] + '_' + str(x[1]) 
                                            if x[1] > 0 else x[0], axis=1)
    new_name = new_name.drop(['index'], axis=1)
    return new_name

In [3]:
import pandas as pd

# 이후 자주 사용되므로 함수로 생성 (레이블 칼럼 명은 'action')
def get_human_dataset():

    # 공백 분리 --> sep으로 할당
    feature_name_df = pd.read_csv(path+'features.txt', sep='\s+', 
                              header=None, 
                              names=['column_index', 'column_name'])
    
    # 중복된 피처명을 처리하는 함수를 이용해서 신규 DataFrame 생성
    new_name = get_new_feature_name_df(feature_name_df)

    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트도 변환
    feature_name = new_name.iloc[:, 1].values.tolist()

    # 학습/테스트 피처 데이터셋 로딩 (칼럼명은 feature_name 적용)
    X_train = pd.read_csv(path+'train/X_train.txt', sep='\s+', 
                          names=feature_name)
    X_test = pd.read_csv(path+'test/X_test.txt', sep='\s+', 
                         names=feature_name)
    
    # 학습/테스트 레이블 데이터셋 로딩 (칼럼명은 'action')
    y_train = pd.read_csv(path+'train/y_train.txt', sep='\s+', 
                          header=None, names=['action'])
    y_test = pd.read_csv(path+'test/y_test.txt', sep='\s+', 
                         header=None, names=['action'])
    
    # 학습/테스트 데이터셋 모두 반환
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_dataset()

In [4]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset()

# GBM 수행 시간 측정을 위한 시작 시간 설정
start_time = time.time()
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_acc = accuracy_score(y_test, gb_pred)

# CPU : 13분
# GPU : 12분
# TPU : 13분
print(f"GBM 정확도 : {gb_acc:.4f}")
print(f"GBM 수행 시간 : {(time.time() - start_time):.2f} 초")

GBM 정확도 : 0.9386
GBM 수행 시간 : 806.11 초


* 기본 하이퍼 파라미터만으로 Random Forest 보다 나은 예측 성능을 보여줍니다.
* 그러나 수행 시간이 오래 걸리고, 하이퍼 파라미터 튜닝 능력도 더 필요합니다.
* 특히 수행 시간은 GBM의 큰 단점입니다.
* 약한 학습기의 순차적인 보정을 통해 학습하므로 멀티 CPU 코어를 사용해도 병렬처리가 지원되지 않아 대용량 데이터의 경우 매우 많은 시간이 걸립니다.
* 느리고 복잡한 GBM보다, 빠르고 쉬운 Random Forest를 많이 사용합니다.

## 5.2 GBM Hyper Parameter Tuning

* `loss='deviance'`
>* 경사하강법에서 사용할 비용 함수를 지정합니다.

* `learning_rate=0.1`
>* GBM 학습 시 적용하는 학습률입니다.
>* 0 ~ 1 사이의 값을 지정할 수 있습니다.
>* 너무 작으면 업데이트되는 값이 작아져 최소 오류값을 찾아 예측 성능이 높아질 수 있습니다.
>* 하지만 많은 약한 학습기는 순차적인 반복이 필요하므로 수행 시간이 오래 걸립니다.
>* 너무 너무 작으면 모든 약한 학습기의 반복이 완료돼도 최소 오류값을 못 찾을 수 있습니다.
>* 너무 크면 최소 오류값을 못 찾고 지나쳐서 예측 성능이 떨어지지만, 빠릅니다.
>* 그래서 `n_estimators`와 상호 보완적으로 조합해 사용합니다.
>* learning_rate < n_estimators : 일정 수준까지 예측 성능이 좋아질 수 있지만 너무 오래 걸리고, 현격히 좋아지진 않습니다.

* `n_estimators=100`
>* 약한 학습기의 개수입니다.
>* 많을 수록 일정 수준까지는 좋아지지만, 오래 걸립니다.

* `subsample=1`
>* 약한 학습기가 학습에 사용하는 데이터의 샘플링 비율입니다.
>* 과적합이 염려되는 경우 1보다 작은 값으로 설정합니다.

* 이제 Grid Search CV로 하이퍼 파라미터 최적화를 진행해 봅시다!
* GBM을 사용하면 시간이 너무 오래 걸려서 간단하게만 비교합니다.
>* `n_estimators` : 100 --> 500
>* `learning_rate` : 0.05 --> 0.1
* 30분 ~ 1시간은 각오합시다!

In [5]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1]}

grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

# CPU : 2시간 30분
# GPU : 실패 ㅠㅠ
# TPU : 2시간 30분
print(f"최적 하이퍼 파라미터 : \n{grid_cv.best_params_}")
print(f"\n최고 예측 정확도 : {grid_cv.best_score_:.4f}")

Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 80.2min finished


최적 하이퍼 파라미터 : 
{'learning_rate': 0.05, 'n_estimators': 500}

최고 예측 정확도 : 0.9013


* 이 최적화된 하이퍼 파라미터 설정을 그대로 테스트셋에 적용해 예측 정확도를 확인합니다.

In [6]:
# Grid Search CV를 이용해 최적으로 학습된 estimator로 예측 수행
gb_pred = grid_cv.best_estimator_.predict(X_test)
gb_acc = accuracy_score(y_test, gb_pred)
print(f"GBM Accuracy : {gb_acc:.4f}")

GBM Accuracy : 0.9396


* GBM은 과적합에도 강하고 뛰어난 예측 성능을 가집니다.
* 하지만 너무 오래 걸립니다.
* GBM 이후 많은 알고리즘이 GBM을 기반으로 새로 만들어지고 있습니다.
* 이 중 ML에서 가장 각광받는 두 개의 Gradient Boosting 기반 패키지가 있습니다.
>* XGBoost
>* LightGBM